<a href="https://colab.research.google.com/github/atotev/ca683-2021-dara/blob/main/Importing_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install requests

In [15]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from statsmodels.compat import lzip
import statsmodels.formula.api as sm
import statsmodels.stats.stattools as st
import statsmodels.stats.api as sms
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [17]:
link ='https://drive.google.com/file/d/1wKZ9-kpdgOUfTt7jGLG1DVuNLdCWvBCz/view?usp=sharing'

In [18]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

sharing


In [19]:
downloaded = drive.CreateFile({'id':'1wKZ9-kpdgOUfTt7jGLG1DVuNLdCWvBCz'}) 

fileMovies = 'movie_titles.csv'

downloaded.GetContentFile(fileMovies)  

movies_netflix = pd.read_csv(fileMovies, encoding = "ISO-8859-1", header = None, names = ['movie', 'year', 'title', 'id_moviedb',	'popularity',	'genre_ids'	,'vote_average',	'media_type'])

#total of 17770



In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
def getFirstPartMovieName(movie):
  if movie.find(":") != -1:
    return movie[0:movie.find(":")]
  if movie.find("(") != -1:
    return movie[0:movie.find("(")]
  if movie.find("/") != -1:
    return movie[0:movie.find("/")]
  return movie

def period_is_ok(release_date, year):
    if (len(release_date)>=4 and len(year) >= 4):
      release_date_int = int(float(release_date[0:4]))
      year_int = int(float(year[0:4]))
      difference = (release_date_int - year_int)
      if (difference >= -2 and difference <= 2 ):
        return True
    return False

In [22]:
from google.colab import files
import requests
from time import sleep
count = 0

for ind in movies_netflix[0:17769].index: 
    count = count + 1

    movie_title = movies_netflix['title'][ind]
    year = (str(movies_netflix['year'][ind])[0:4])

    movie_title_part = getFirstPartMovieName(movie_title)

    url = "https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query='" + str(movie_title_part) +"'&year='" + year + "'"
    response = requests.get(url)
    results = response.json()

    movie = {}

    print(url)

    if (len(results["results"])) >= 1:
      for item in results["results"]:
        if item["media_type"] != "person":
          if "name" in item:            
            if item["name"] == movie_title:
              movie = item
              break
          if "title" in item: 
            if item["title"] == movie_title:
              movie = item
              break
          if "original_title" in item:             
            if item["original_title"] == movie_title:
              movie = item
              break
          if "release_date" in item:                   
            if period_is_ok(item["release_date"], year):
              movie = item
              break
          if "first_air_date" in item:    
            if period_is_ok(item["first_air_date"], year):
              movie = item
              break
             
    elif (len(results["results"])) == 1:
      movie = results["results"][0]
        
    if movie:
      movies_netflix['id_moviedb'][ind] = movie["id"]
      if "genre_ids" in movie:
        movies_netflix['genre_ids'][ind] = movie["genre_ids"]
      if "vote_average" in movie:
        movies_netflix['vote_average'][ind] = movie["vote_average"]
      if "media_type" in movie:
        movies_netflix['media_type'][ind] = movie["media_type"]
      if "popularity" in movie:
        movies_netflix['popularity'][ind] = movie["popularity"]
      #if "release_date" in movie:
        #movies_netflix['release_date'][ind] = movie["release_date"]
      print(movie_title + " movie found")
    else:
      print(movie_title + " not found")

    if count == 10:
      movies_netflix.to_csv('movie_titles.csv', header=True)
      !cp movie_titles.csv "gdrive/My Drive/"
      count = 0
      sleep(1.5)


https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query='Dinosaur Planet'&year='2003'
Dinosaur Planet movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query='Isle of Man TT 2004 Review'&year='2004'
Isle of Man TT 2004 Review not found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query='Character'&year='1997'
Character movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query='Paula Abdul's Get Up & Dance'&year='1994'
Paula Abdul's Get Up & Dance movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query='The Rise and Fall of ECW'&year='2004'
The Rise and Fall of ECW movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query='Sick'&year='1997'
Sick movie foun

KeyboardInterrupt: ignored

In [ ]:



movies_netflix.to_csv('movie_titles.csv', header=True)
!cp movie_titles.csv "gdrive/My Drive/"

In [ ]:
movies_netflix[movies_netflix["genre_ids"] == "[]" ].count

#movies_netflix[(movies_netflix["genres"].notnull()) ]
